In [1]:
%pip install psycopg2-binary
%pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np

# Load the .env file
load_dotenv()

connection_string = os.getenv("CONNECTION_STRING")

with psycopg.connect(connection_string) as conn:
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the query
    query = "SELECT * FROM sites"
    cursor.execute(query)

    # Fetch all the results
    results = cursor.fetchall()

    # Get the column names from cursor.description
    column_names = [desc[0] for desc in cursor.description]

    # Close the cursor
    cursor.close()

    # No need to manually close the connection when using 'with'

    # Convert the results to a data frame with column names
    df_sites = pd.DataFrame(results, columns=column_names)
    df_sites.columns = [column_names]



In [4]:
import pandas as pd
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

from dotenv import load_dotenv

# Load the .env file
load_dotenv()

user = os.environ.get("USER")
pw = os.environ.get("PASS")
db = os.environ.get("DB")
host = os.environ.get("HOST")
api = os.environ.get("API")
port = os.environ.get("PORT")

uri = f"postgresql+psycopg2://{quote_plus(user)}:{pw}@{host}:{port}/{db}"
alchemyEngine = create_engine(uri)

q = """SELECT sites.uuid AS site_uuid, sites.short_name AS site_short_name, sites.name AS site_name, releases.uuid AS release_uuid, releases.release_date, releases.short_name AS release_short_name, releases.name AS release_name, releases.url AS release_url FROM releases JOIN sites ON releases.site_uuid = sites.uuid WHERE sites.short_name = 'wowgirls';"""

dbConnection = alchemyEngine.connect()
df = pd.read_sql(q, dbConnection)
df["release_date"] = pd.to_datetime(df["release_date"])

df_culture_extractor_scenes = df[df['site_short_name'] == 'wowgirls'][df['release_url'].str.contains('/film/', case=False, na=False)]
df_culture_extractor_scenes

,site_uuid,site_short_name,site_name,release_uuid,release_date,release_short_name,release_name,release_url
0,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-78e2-71f9-8e48-f43312f94ca0,2020-12-11,g9f65848,She Wants You,https://venus.wowgirls.com/film/g9f65848/she-w...
16,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-9367-712d-a65d-a6a916638669,2014-10-21,fb771eb5,Your Beauty Sucks Me In,https://venus.wowgirls.com/film/fb771eb5/your-...
18,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-8f05-73f1-9e31-646631b53434,2016-03-11,w7143c67,Dirty Minds Think Alike,https://venus.wowgirls.com/film/w7143c67/dirty...
20,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-8f05-743a-bdee-6b37dd775f33,2015-06-21,e5be54e1,One Thing On My Mind,https://venus.wowgirls.com/film/e5be54e1/one-t...
21,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-8f05-741d-b4d2-8f8adb5abbab,2015-10-29,ic68818f,On Cloud Nine,https://venus.wowgirls.com/film/ic68818f/on-cl...
...,...,...,...,...,...,...,...,...
4126,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-78e2-71fb-adba-83ce710c836c,2020-11-27,e432c030,"I Like To Play, To Tease",https://venus.wowgirls.com/film/e432c030/i-lik...
4131,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-7469-7848-af51-e28e4259f196,2021-06-25,wd069cfa,Gets Hot With Rough,https://venus.wowgirls.com/film/wd069cfa/gets-...
4132,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-7469-7831-bfda-9df00c33ce96,2021-09-17,c11cfac9,Perfect Girls For Threesome,https://venus.wowgirls.com/film/c11cfac9/perfe...
4135,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-5ad4-7328-b2d1-0f573bebc569,2022-03-19,j0805f91,Interview With Alecia Fox,https://venus.wowgirls.com/film/j0805f91/inter...


In [5]:
import pandas as pd
import stashapi.log as log
from stashapi.stashapp import StashInterface
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key
api_key = os.getenv("ApiKey")

stash = StashInterface(
    {
        "scheme": "http",
        "host": "fraktal.auvilankuja.fi",
        "port": "9999",
        "logger": log,
        "ApiKey": api_key
    }
)


d Using stash (v0.25.1-0) endpoint at http://fraktal.auvilankuja.fi:9999/graphql


In [6]:
def get_parent_studio_id(studio):
    if studio["parent_studio"] is not None:
        return studio["parent_studio"]["id"]
    return None

studios = stash.find_studios({})
df_stash_studios = pd.DataFrame(studios)
df_stash_studios["parent_studio_id"] = df_stash_studios.apply(get_parent_studio_id, axis=1)


In [17]:
# Lookup functions
def get_studio(studio_id):
    return df_stash_studios[df_stash_studios["id"] == studio_id]


In [18]:
current_studio = get_studio("2")
current_studio

,id,name,url,parent_studio,child_studios,aliases,ignore_auto_tag,image_path,scene_count,image_count,gallery_count,performer_count,movie_count,stash_ids,rating100,details,created_at,updated_at,movies,parent_studio_id
883,2,WowGirls,https://www.wowgirls.com/,{'id': '221'},[],[],False,http://fraktal.auvilankuja.fi:9999/studio/2/im...,311,0,0,142,0,"[{'endpoint': 'https://stashdb.org/graphql', '...",NaN,,2023-01-25T13:04:53Z,2024-04-21T11:17:27Z,[],221


In [40]:
df_stash_scenes = pd.DataFrame(stash.find_scenes(
    {
        "studios": { "value": ["2"], "excludes": [], "modifier": "INCLUDES" }
    },
    filter={
        "per_page": 500, "page": 1, "sort": "path", "direction": "DESC"
    }
))

def get_endpoint_stash_id(stash_ids, endpoint):
    for stash_id in stash_ids:
        if stash_id["endpoint"] == endpoint:
            return stash_id["stash_id"]
    return None

def get_tpdb_id(stash_ids):
    return get_endpoint_stash_id(stash_ids, "https://theporndb.net/graphql")

def get_stashdb_id(stash_ids):
    return get_endpoint_stash_id(stash_ids, "https://stashdb.org/graphql")

def get_culture_extractor_id(stash_ids):
    return get_endpoint_stash_id(stash_ids, "https://culture.extractor/graphql")

df_stash_scenes["date"] = pd.to_datetime(df_stash_scenes["date"])
df_stash_scenes["tpdb_id"] = df_stash_scenes["stash_ids"].apply(get_tpdb_id)
df_stash_scenes["stashdb_id"] = df_stash_scenes["stash_ids"].apply(get_stashdb_id)
df_stash_scenes["culture_extractor_id"] = df_stash_scenes["stash_ids"].apply(get_culture_extractor_id)

df_stash_scenes.head()

,id,title,code,details,director,urls,date,rating100,organized,o_counter,...,galleries,studio,movies,tags,performers,stash_ids,sceneStreams,tpdb_id,stashdb_id,culture_extractor_id
0,16909,Interview with Alissa Foxy,h0cdfb3d,,,[https://venus.wowgirls.com/film/h0cdfb3d/inte...,2024-03-02,None,False,0,...,[],{'id': '2'},[],[],[{'id': '251'}],[{'endpoint': 'https://culture.extractor/graph...,[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,877650fc-edf6-44f1-b9a9-39e8d4a7b04c,ee410059-73b5-4c6b-8fe1-cafb2c77cc2d,018e0820-46a3-733f-bb1b-69e67efad5ad
1,16908,Starlets in Love,d468d36d,,,[https://venus.wowgirls.com/film/d468d36d/star...,2023-12-24,None,False,0,...,[],{'id': '2'},[],[],"[{'id': '230'}, {'id': '251'}, {'id': '312'}]",[{'endpoint': 'https://culture.extractor/graph...,[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,b69886d4-019f-4a23-9a1e-e7c9d08c0ee0,8548fce4-8778-48b1-935f-e76a5df3dbcd,018dbacf-8718-7746-ae8a-5d6cd191d5e6
2,16907,Cool Girlfriends,u9816e30,,,[https://venus.wowgirls.com/film/u9816e30/cool...,2023-10-03,None,False,0,...,[],{'id': '2'},[],[],"[{'id': '251'}, {'id': '640'}]",[{'endpoint': 'https://culture.extractor/graph...,[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,None,9d9c36bb-af9f-4e0c-81aa-fc6115d88bd6,018dbad0-afc1-76a0-b14e-b335b49ad0ad
3,16906,Wet Dreams,b5c0ee46,,,[https://venus.wowgirls.com/film/b5c0ee46/wet-...,2023-07-25,None,False,0,...,[],{'id': '2'},[],[],"[{'id': '251'}, {'id': '640'}]",[{'endpoint': 'https://culture.extractor/graph...,[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,None,6751a479-644b-4561-af8b-5bb30f0d909b,018dbad1-8ba8-74cd-937a-6d444e0da24b
4,16905,Intimate Bond,x4bd4882,,,[https://venus.wowgirls.com/film/x4bd4882/inti...,2023-07-22,None,False,0,...,[],{'id': '2'},[],[],[{'id': '251'}],[{'endpoint': 'https://culture.extractor/graph...,[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,None,6c48d8b0-9db0-44fd-b69a-06e9db9a3ff9,018dbad1-9c29-736e-9df3-8e9fe8e51671


In [9]:
df_merged_scenes = df_stash_scenes.merge(df_culture_extractor_scenes, left_on='date', right_on='release_date', how='left')
df_merged_scenes

,id,title,code,details,director,urls,date,rating100,organized,o_counter,...,stash_ids,sceneStreams,site_uuid,site_short_name,site_name,release_uuid,release_date,release_short_name,release_name,release_url
0,16909,Interview with Alissa Foxy,h0cdfb3d,,,[https://venus.wowgirls.com/film/h0cdfb3d/inte...,2024-03-02,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018e0820-46a3-733f-bb1b-69e67efad5ad,2024-03-02,h0cdfb3d,Interview With Alissa Foxy,https://venus.wowgirls.com/film/h0cdfb3d/inter...
1,16908,Starlets in Love,d468d36d,,,[https://venus.wowgirls.com/film/d468d36d/star...,2023-12-24,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018dbacf-8718-7746-ae8a-5d6cd191d5e6,2023-12-24,d468d36d,Starlets In Love,https://venus.wowgirls.com/film/d468d36d/starl...
2,16907,Cool Girlfriends,u9816e30,,,[https://venus.wowgirls.com/film/u9816e30/cool...,2023-10-03,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018dbad0-afc1-76a0-b14e-b335b49ad0ad,2023-10-03,u9816e30,Cool Girlfriends,https://venus.wowgirls.com/film/u9816e30/cool-...
3,16906,Wet Dreams,b5c0ee46,,,[https://venus.wowgirls.com/film/b5c0ee46/wet-...,2023-07-25,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018dbad1-8ba8-74cd-937a-6d444e0da24b,2023-07-25,b5c0ee46,Wet Dreams,https://venus.wowgirls.com/film/b5c0ee46/wet-d...
4,16905,Intimate Bond,x4bd4882,,,[https://venus.wowgirls.com/film/x4bd4882/inti...,2023-07-22,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018dbad1-9c29-736e-9df3-8e9fe8e51671,2023-07-22,x4bd4882,Intimate Bond,https://venus.wowgirls.com/film/x4bd4882/intim...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,16221,Let's Come Together,xf5ff9ae,Group sex has always been one of those things ...,,[https://wowgirlsfan.com/wow-girls-klara-palom...,2012-04-12,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-a528-7630-ad4c-70871b50a058,2012-04-12,xf5ff9ae,Let's Come Together,https://venus.wowgirls.com/film/xf5ff9ae/let-c...
307,787,Sharing Is Caring,ma772cd8,Anjelica lies nude and asleep next to her nake...,,[https://wowgirlsfan.com/wow-girls-anjelica-pe...,2012-04-07,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-a528-7632-adb2-b7a2b59d4802,2012-04-07,ma772cd8,Sharing Is Caring,https://venus.wowgirls.com/film/ma772cd8/shari...
308,904,Born To Please Part 2,qccfe718,Beata wears a sexy summer dress that opens wid...,,[https://wowgirlsfan.com/wow-girls-beata-in-bo...,2012-04-01,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-a528-7635-a3a7-71c7c995a9da,2012-04-01,qccfe718,Born To Please 2,https://venus.wowgirls.com/film/qccfe718/born-...
309,16220,DJ Mia,y675700d,Mia was practicing for her DJ gig later that n...,,[https://venus.wowgirls.com/film/y675700d/dj-m...,2012-03-16,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-a528-763b-a13b-5e7904c64937,2012-03-16,y675700d,DJ Mia,https:/

In [10]:
df_unmatched_scenes = df_merged_scenes[df_merged_scenes["release_uuid"].isnull()]
df_unmatched_scenes

,id,title,code,details,director,urls,date,rating100,organized,o_counter,...,stash_ids,sceneStreams,site_uuid,site_short_name,site_name,release_uuid,release_date,release_short_name,release_name,release_url


In [47]:
unmatched_scenes_ids = df_unmatched_scenes["id"].unique()

In [49]:
df_merged_unmatched_scenes = df_stash_scenes[df_stash_scenes["id"].isin(unmatched_scenes_ids)].merge(df_culture_extractor_scenes, left_on='title', right_on='release_name', how='left')

In [12]:
df_merged_scenes[df_merged_scenes["code"] != df_merged_scenes["release_short_name"]][["id", "title", "code", "urls", "date", "release_uuid", "release_date", "release_short_name", "release_name", "release_url"]]

,id,title,code,urls,date,release_uuid,release_date,release_short_name,release_name,release_url


In [50]:
df_merged_unmatched_scenes

,id,title,code,details,director,urls,date,rating100,organized,o_counter,...,stash_ids,sceneStreams,site_uuid,site_short_name,site_name,release_uuid,release_date,release_short_name,release_name,release_url
0,5422,Eternal Desire,,Softcore touching as natural beauty Alissa Fox...,,"[https://wowgirlsblog.com/eternal-desire-2, ht...",2022-01-25,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-7469-77ff-8dee-083897dcc93c,2022-01-26,d511222f,Eternal Desire,https://venus.wowgirls.com/film/d511222f/etern...
1,791,The Passion For Sex,,"Everyone knows Anjelica by now, and if they do...",,[https://wowgirls.tv/videos/anjelica/the-passi...,2014-11-30,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-9367-711e-b403-3221c2f39bd1,2014-12-04,v34d34c0,The Passion For Sex,https://venus.wowgirls.com/film/v34d34c0/the-p...
2,799,Daytime Means Playtime,,,,[https://www.iafd.com/title.rme/title=daytime+...,2014-10-25,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-9367-712a-9fa0-caeb4c1695f5,2014-10-30,u1f0939c,Daytime Means Playtime,https://venus.wowgirls.com/film/u1f0939c/dayti...
3,797,Refreshing Morning,,,,[https://www.iafd.com/title.rme/title=refreshi...,2013-12-06,None,False,0,...,"[{'endpoint': 'https://stashdb.org/graphql', '...",[{'url': 'http://fraktal.auvilankuja.fi:9999/s...,018b94b1-b5e9-71be-870e-b479d427446c,wowgirls,WowGirls,018b8e89-97d9-772b-8952-e9ae2f64d3e9,2014-01-04,m8a2bbb1,Refreshing Morning,https://venus.wowgirls.com/film/m8a2bbb1/refre...


In [52]:
for index, row in df_merged_scenes.iterrows():
    id = row["id"]
    release_uuid = row["release_uuid"]
    stash.update_scene({ "id": id, "code": code, "date": release_date })

In [141]:
# Use the `apply` function with a lambda to check if `release_url` is not in `urls` for each row
mask = df_merged_scenes.apply(lambda row: row['release_url'] not in row['urls'], axis=1)

# Use the mask to filter the DataFrame
df_culture_extractor_scenes = df_merged_scenes[mask][["id", "title", "code", "urls", "date", "release_uuid", "release_date", "release_short_name", "release_name", "release_url"]]


In [24]:
# Create a new column 'new_urls' where 'release_url' is appended to 'urls' if not already present
df_merged_scenes['new_stash_ids'] = df_merged_scenes.apply(lambda row:
                                                           row['stash_ids'] + [{ "endpoint": "https://culture.extractor/graphql", "stash_id": str(row["release_uuid"]) }] if "https://culture.extractor/graphql" not in [stash_id_obj["endpoint"] for stash_id_obj in row['stash_ids']] else row['stash_ids'],
                                                           axis=1)


In [25]:
df_merged_scenes[["new_stash_ids"]].values

array([[list([{'endpoint': 'https://stashdb.org/graphql', 'stash_id': 'ee410059-73b5-4c6b-8fe1-cafb2c77cc2d'}, {'endpoint': 'https://theporndb.net/graphql', 'stash_id': '877650fc-edf6-44f1-b9a9-39e8d4a7b04c'}, {'endpoint': 'https://culture.extractor/graphql', 'stash_id': '018e0820-46a3-733f-bb1b-69e67efad5ad'}])],
       [list([{'endpoint': 'https://stashdb.org/graphql', 'stash_id': '8548fce4-8778-48b1-935f-e76a5df3dbcd'}, {'endpoint': 'https://theporndb.net/graphql', 'stash_id': 'b69886d4-019f-4a23-9a1e-e7c9d08c0ee0'}, {'endpoint': 'https://culture.extractor/graphql', 'stash_id': '018dbacf-8718-7746-ae8a-5d6cd191d5e6'}])],
       [list([{'endpoint': 'https://stashdb.org/graphql', 'stash_id': '9d9c36bb-af9f-4e0c-81aa-fc6115d88bd6'}, {'endpoint': 'https://culture.extractor/graphql', 'stash_id': '018dbad0-afc1-76a0-b14e-b335b49ad0ad'}])],
       [list([{'endpoint': 'https://stashdb.org/graphql', 'stash_id': '6751a479-644b-4561-af8b-5bb30f0d909b'}, {'endpoint': 'https://culture.extractor/

In [36]:
for index, row in df_merged_scenes.iterrows():
    id = row["id"]
    # old_urls = row["urls"]
    # release_url = row["release_url"]
    # new_urls = row["new_urls"]
    stash_ids = row["stash_ids"]
    new_stash_ids = row["new_stash_ids"]
    stash_ids_set = set(stash_id_obj["stash_id"] for stash_id_obj in row["stash_ids"])
    new_stash_ids_set = set(stash_id_obj["stash_id"] for stash_id_obj in row["new_stash_ids"])
    if stash_ids_set != new_stash_ids_set:
        print(row["title"])
        print(stash_ids_set)
        print(new_stash_ids_set)
        stash.update_scene({ "id": id, "stash_ids": new_stash_ids })

Interview with Alissa Foxy
{'877650fc-edf6-44f1-b9a9-39e8d4a7b04c', 'ee410059-73b5-4c6b-8fe1-cafb2c77cc2d'}
{'877650fc-edf6-44f1-b9a9-39e8d4a7b04c', 'ee410059-73b5-4c6b-8fe1-cafb2c77cc2d', '018e0820-46a3-733f-bb1b-69e67efad5ad'}
Starlets in Love
{'b69886d4-019f-4a23-9a1e-e7c9d08c0ee0', '8548fce4-8778-48b1-935f-e76a5df3dbcd'}
{'018dbacf-8718-7746-ae8a-5d6cd191d5e6', 'b69886d4-019f-4a23-9a1e-e7c9d08c0ee0', '8548fce4-8778-48b1-935f-e76a5df3dbcd'}
Cool Girlfriends
{'9d9c36bb-af9f-4e0c-81aa-fc6115d88bd6'}
{'018dbad0-afc1-76a0-b14e-b335b49ad0ad', '9d9c36bb-af9f-4e0c-81aa-fc6115d88bd6'}
Wet Dreams
{'6751a479-644b-4561-af8b-5bb30f0d909b'}
{'018dbad1-8ba8-74cd-937a-6d444e0da24b', '6751a479-644b-4561-af8b-5bb30f0d909b'}
Intimate Bond
{'6c48d8b0-9db0-44fd-b69a-06e9db9a3ff9'}
{'6c48d8b0-9db0-44fd-b69a-06e9db9a3ff9', '018dbad1-9c29-736e-9df3-8e9fe8e51671'}
Kinky Moods
{'aa96d278-36b9-428f-bcec-cd1f3a29fe9e'}
{'018dbad2-b254-7497-91a6-8596074de0c7', 'aa96d278-36b9-428f-bcec-cd1f3a29fe9e'}
Legs Wide 